# Laboratorio 4
## Clasificación de malware
Detección de malware utilizando métodos de ML
> #### Gerardo Méndez 18239

In [1]:
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
## cargamos el set de datos
df = pd.read_csv('datasets/virus_sample.csv')

### Parte 1 - Ingeniería de Características

#### Exploración de datos

In [3]:
## mostramos los datos
df.head(10)

,file,api,class
0,7ff49f2f0912352416b05c010f35f402cc79feed,"IntersectRect,GetCurrentProcess,GetVersion",Virus
1,50cc6c99ec285d0db45dde07d8fdc18d9098c5b6,"GetCaretBlinkTime,CountClipboardFormats,GetCon...",Virus
2,f77c6bd4aebacd1a01d02e0cb20642ebf2d32929,"VarR8Pow,GetClipboardViewer,GetInputDesktop,Ge...",Virus
3,349c367c5b88fbb6cafae5d7109588d7250e16b5,"SetTraceCallback,CopyAcceleratorTableW,GetProc...",Virus
4,021f4aa86b520e1d606ab26699c35546bcd00c27,"SHLoadNonloadedIconOverlayIdentifiers,VarUI8Fr...",Virus
5,124491349667be19c61a5f938f139eb326b9223b,"NtReplyWaitReceivePortEx,NtSetInformationJobOb...",Virus
6,6aea319c23473d1b837972a064bc20d79404c859,"SHGetNewLinkInfoA,GetSystemDefaultUILanguage,L...",Virus
7,7682fa3c7f4ecbd9df428630778b6662ea172e71,"CryptGenRandom,GetDoubleClickTime,GetDialogBas...",Virus
8,1c0d53d5ed17b9f68f6c1c2fc034e766380a1665,"GetVersion,GetTapePosition,GetLastError,SetMen...",Virus
9,2d25cdbd7b6ea9a03b270525d13eda302be3edc1,"VarUI8FromStr,SetVarConversionLocaleSetting,Nt...",Virus


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9795 entries, 0 to 9794
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   file    9795 non-null   object
 1   api     9795 non-null   object
 2   class   9795 non-null   object
dtypes: object(3)
memory usage: 229.7+ KB


In [6]:
df['class'].value_counts()

Trojan        6153
Virus         2367
Backdoor       447
Worms          441
Adware         222
Agent          102
Downloader      31
Spyware         11
Ransomware      10
Riskware         4
Dropper          4
Crypt            2
Keylogger        1
Name: class, dtype: int64

In [7]:
df['file'].value_counts()

7ff49f2f0912352416b05c010f35f402cc79feed                            1
67a61c136413b58355038987138e621e0aba4ef0                            1
09934398570d913cf2feffa410eef8774a4ba327                            1
c88df225adfbe5179ced40655d0db6163e03e924                            1
5c0ba959963cdb845c8b04de1ae87594b03bd7b4                            1
                                                                   ..
08c56708fcf139be7d83b19e9d4ea2f2b0358bf2                            1
1cf5dd8b58466c066508963d583039133efc1473                            1
1754f3b202bd9524dd5954bb5497023544969e39                            1
1e7a91da3b78a7aeba8213e2ad3eb67a7125e2ad                            1
be7d4be8e48719257d6040c3e9a1f6f15bc2616ca2227657f2f095af20bc7100    1
Name: file, Length: 9795, dtype: int64

#### Preprocesamiento

In [13]:
label = 'class'

In [14]:
X = df.drop(columns=label, axis=1)

In [17]:
y = df[label]

In [18]:
y

0        Virus
1        Virus
2        Virus
3        Virus
4        Virus
         ...  
9790    Trojan
9791    Trojan
9792    Trojan
9793    Trojan
9794    Trojan
Name: class, Length: 9795, dtype: object

#### División de los datos

In [22]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [23]:
test_ratio = 0.3

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio)

**Modelo 1**

In [ ]:
bog_classification_report = classification_report(bog_y_test, bog_y_pred)
print(bog_classification_report)

**Modelo 2**

In [ ]:
bayes.fit(tf_x_train, tf_y_train)

In [ ]:
tf_y_pred = bayes.predict(tf_x_test)

tf_cf_matrix = confusion_matrix(tf_y_test, tf_y_pred)
tf_cf_matrix

In [ ]:
classification_report = classification_report(tf_y_test, tf_y_pred)
print(classification_report)

### Conclusion

Compare los valores para cada representación numérica. ¿Qué representación numérica produjo el 
mejor resultado? Explique los valores obtenidos en las métricas para el mejor modelo. 